# Data Mining Final Project
### Yazdan asadi 



## Exercise 1: 

In [1]:
import pandas as pd
import numpy as np
from os import walk
import math 
import random
import sets
import time
from IPython.display import Image
import uuid
import pydot
from sklearn.model_selection import train_test_split
import graphviz 
import pprint
import matplotlib.pyplot as plt
import six
from random import randrange


In [3]:
path = '/home/yazdan/Desktop/DM_Project'
df_dic = {}
for root, dirs, files in walk(path):
    for file in files:
        if root not in df_dic:
            df_dic[root] = [file]
        else:
            df_dic[root].append(file)

after we put all subfolders in the dictionary we choose to get datas in each folder to a dataframe

In [4]:
dfs = []
for i in df_dic:
    for f in df_dic[i]:
        if f.split('.')[-1] in ['trn','tst','data']:
            full_path = i + '/' + f
            dfs.append(full_path)

In [5]:
dfs

['/home/yazdan/Desktop/DM_Project/1-Balance Scale Database/balance-scale.data',
 '/home/yazdan/Desktop/DM_Project/2-Balloons Database/yellow-small.data',
 '/home/yazdan/Desktop/DM_Project/2-Balloons Database/adult+stretch.data',
 '/home/yazdan/Desktop/DM_Project/2-Balloons Database/adult-stretch.data',
 '/home/yazdan/Desktop/DM_Project/2-Balloons Database/yellow-small+adult-stretch.data',
 '/home/yazdan/Desktop/DM_Project/8-Adult Database/adult.data',
 '/home/yazdan/Desktop/DM_Project/7-Event Detection Database/CalIt2.data',
 '/home/yazdan/Desktop/DM_Project/7-Event Detection Database/Dodgers.data',
 '/home/yazdan/Desktop/DM_Project/6-Dermatology Database/dermatology.data',
 '/home/yazdan/Desktop/DM_Project/9-Abalone Database/abalone.data',
 '/home/yazdan/Desktop/DM_Project/4-Credit Screening Databases/crx.data',
 '/home/yazdan/Desktop/DM_Project/5-Cylinder Bands Database/bands.data',
 '/home/yazdan/Desktop/DM_Project/3-Wisconsin Breast Cancer Databases/wpbc.data',
 '/home/yazdan/Deskt

In [6]:
baloons_df_paths = dfs[1] +','+ dfs[2] +','+ dfs[3]

In [7]:
balance_df_path = dfs[0]

In [ ]:
breast_df_paths = dfs[12] +','+ dfs[13] + ',' +dfs[14]

In [ ]:
crx_df_path = dfs[10]

In [ ]:
satimage_df_paths = dfs[15] +','+ dfs[16]

In [ ]:
abalone_df_path = dfs[9]

In [ ]:
adult_df_path = dfs[5]

Now we found 7 datasets including test and train csv files. its time to read this csv files 

In [ ]:
datas = []
df = []
data_to_csv = balance_df_path #enter your desire dataset path
datas = data_to_csv.split(",")
for i in datas:
    df = pd.read_csv(i,sep=',')
    

In [ ]:
df.head(200)

##### EDA and Preprocessing

step 1: Base on the problem and dataset we can drop the unnecessary columns!
For each dataset i made a "to_drop" list for droping the cols.

In [ ]:
to_drop = [] #FILL THE LIST WITH COLS THAT YOU WANT!
class_labels = ['Class-Name']
df.drop(to_drop, inplace=True, axis=1)

step 2: Checking if the class label(s) is unique or not! and checking the values.

In [ ]:
df['Class-Name'].unique()

In [ ]:
col_names = df.columns
for i in col_names:
      print('{} is unique: {}'.format(i, df[i].is_unique))

step 3: Checking the missing values and number of dirty values!

In [ ]:
#Before (shape of df)
df.shape

In [ ]:
#after (shape of df)
rows_without_missing_data = df.dropna()
rows_without_missing_data.shape

In [ ]:
data_without_missing_columns = df.dropna(axis=1)
data_without_missing_columns.shape

In [ ]:
print(df.isnull().sum())

In [ ]:
df.duplicated().sum()

In [ ]:
# Fill NaN with ' '
#df['col'] = df['col'].fillna(' ')
# Fill NaN with 99
#df['col'] = df['col'].fillna(99)
# Fill NaN with the mean of the column
#df['col'] = df['col'].fillna(df['col'].mean())

step 4: time to look at the information and describtion of our dataframe!

In [ ]:
df.info()

In [ ]:
df.describe()

step 5: Replacing the Classes with numbers and then we have label (y) for training!

In [ ]:
class_to_num = {'B': 0,
                  'R': 1,
                  'L': 2}
df['tmp'] = df['Class-Name'].map(class_to_num)
y = df['tmp']

In [ ]:
y

## Exercise 2: imbalanced data

For checking if our data is imbalance or not we can use shannon entroy equation:                              
0 for a unbalanced data set                                             
1 for a balanced data set

In [ ]:
from scipy import stats
import itertools

In [ ]:
pd_series = pd.Series(df.all)
counts = pd_series.value_counts()
entropy = stats.entropy(counts)
print(entropy)

## Exercise 5: Train/Test Split without sklearn and Producing Decision Tree!

In [ ]:
df2 = df.copy()
df2.drop(class_labels,inplace=True, axis=1)
np.random.seed(43)
indices = list(range(df2.shape[0]))
num_training_indices = int(0.66 * df2.shape[0])
np.random.shuffle(indices)
train_indices = indices[:num_training_indices]
test_indices = indices[num_training_indices:]
# split the actual data
x_train, x_test = df2.iloc[train_indices], df2.iloc[test_indices]
y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

### *time to make our decision tree*

1.compute the entropy for data-set                      
2.for every attribute/feature:
                  1.calculate entropy for all categorical values
               2.take average information entropy for the current attribute
         3.calculate gain for the current attribute
3. pick the highest gain attribute.
4. Repeat until we get the tree we desired

In [ ]:
def build_tree(data,y_field,level=0,tree=None,verbose=False):
    cols = list(data.columns)
    cols.remove(y_field)
    info_gains = []
    for col in cols:
        info_gains.append(round(info_gain(data,col,y_field),2))
    if verbose:
        print(cols,info_gains)
    if(len(set(info_gains))==1): 
        col_to_split_on = random.choice(cols)        
        values_to_split_on = data[col_to_split_on].unique()
        node = col_to_split_on
        if tree is None:
            tree = {}
            tree[node] = {}
        for value in values_to_split_on:
            sub_df = data[data[col_to_split_on]==value]
            if verbose:
                print(f'No info gain, uniques: {sub_df[y_field].unique()}')
            most_frequent = sub_df[y_field].mode()[0]
            if verbose:
                print(f'Setting {col_to_split_on} with value {value} as leaf with output {most_frequent}')
            tree[node][str(value)] = str(most_frequent)
    else:
        col_to_split_on = cols[info_gains.index(max(info_gains))]
        values_to_split_on = data[col_to_split_on].unique()
        node = col_to_split_on
        if tree is None:
            tree = {}
            tree[node] = {}
        for value in values_to_split_on:
            sub_df = data[data[col_to_split_on]==value]
            if verbose:
                print(f'Level->{level},total_cols:{cols}, Split on {col_to_split_on} on value {value}, uniques:{sub_df[y_field].unique()}')
            if len(sub_df[y_field].unique())==1:
                tree[node][str(value)] = str(sub_df[y_field].unique()[0])
                if verbose:
                    print(f'Level->{level}, Reached leaf for {col_to_split_on} at value {value} with output {sub_df[y_field].unique()[0]}')
            else:
                x = 'y'
                if x !='n':
                    tree[node][str(value)] = build_tree(sub_df,y_field,level=level+1,verbose=verbose)
                else:
                    tree[node][str(value)]=None
    return tree

In [ ]:
def entropy(data,y_field):
    sum = 0
    for el in data[y_field].value_counts():
        prob = el/len(data)
        sum += prob*math.log(prob,2)
    sum*=-1
    return sum

In [ ]:
def info_gain(data,attribute,y_field):
    current_entropy = entropy(data,y_field)
    uniques = data[attribute].value_counts().keys()
    sec_term = 0
    for i,cat_count in enumerate(data[attribute].value_counts()):
        sec_term += (cat_count/len(data))*entropy(data[data[attribute]==uniques[i]],y_field)
    entropy_for_attribute = current_entropy - sec_term
    return entropy_for_attribute


In [ ]:
my_tree = build_tree(x_train,'tmp',verbose=True)

In [ ]:
def walk_dictionaryv2(graph, dictionary, parent_node=None):
    for k in dictionary.keys():

        if parent_node is not None:

            from_name = parent_node.get_name().replace("\"", "") + '_' + str(k)
            from_label = str(k)
            obj_dict = {}

            if 'True' in from_label:
                node_from = pydot.Node(from_name, color='white', style='filled',fillcolor='green', label=from_label)
            elif 'False' in from_label:
                node_from = pydot.Node(from_name, color='white', style='filled',fillcolor='red', label=from_label)
            else:
                node_from = pydot.Node(from_name, label=from_label)

            graph.add_node(node_from)

            graph.add_edge( pydot.Edge(parent_node, node_from) )

            if isinstance(dictionary[k], dict): # if interim node
                walk_dictionaryv2(graph, dictionary[k], node_from)

            else: # if leaf node
                to_name = str(uuid.uuid4()) + '_' + str(dictionary[k]) # unique name
                to_label = str(dictionary[k])

                node_to = pydot.Node(to_name, label=to_label, shape='box')
                graph.add_node(node_to)
                graph.add_edge(pydot.Edge(node_from, node_to))
        else:

            from_name =  str(k)
            from_label = str(k)

            node_from = pydot.Node(from_name, label=from_label)
            walk_dictionaryv2(graph, dictionary[k], node_from)


def plot_tree(tree, name):
    graph = pydot.Dot(graph_type='graph')
    walk_dictionaryv2(graph, tree)
    graph.write_png(name+'.png')

In [ ]:
plot_tree(my_tree,'first')
Image(filename='first.png',unconfined=True) 

### Comparing the model with sklearn classifier 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf_ent = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
# fit the model
clf_ent.fit(x_train, y_train)

In [ ]:
y_pred_ent = clf_ent.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_ent))

## Exercise 8 : APRIOR / FP-Growth for freq-itemsets

In [ ]:
import itertools as itert
from itertools import combinations
import operator

In [ ]:
datas = []
df = []
data_to_csv = adult_df_path #enter your desire dataset path
datas = data_to_csv.split(",")
for i in datas:
    df = pd.read_csv(i,sep=',')

In [ ]:
df.head(50)

In [ ]:
def preprocessData(df):
    
    # MISSING VALUES
    df = df.applymap(lambda d: np.nan if d== ' ?' else d)
    df = df.dropna(axis = 0) # Drop Rows With NaNs
    
    # DROP UNNESSESARY COLS
    df.drop('fnlwgt', axis = 1, inplace = True)
    df.drop('educational-num', axis = 1, inplace = True)
    
    # Converting Continuous Variables to Categorical
    df['age'] = pd.cut(df['age'], [0, 30, 60, 100], 
                labels = ['Young', 'Middle-aged', 'Senior'], 
                right = True, include_lowest = True)
    df['hours-per-week'] = pd.cut(df['hours-per-week'], [0, 20, 40, 100], 
                           labels = ['Part-time', 'Full-time', 'Over-time'], 
                           right = True, include_lowest = True)
    df['capital-gain'] = pd.cut(df['capital-gain'], [0, 1, 100000], 
                       labels = ['No-Gain', 'Positive-Gain'], 
                       right = True, include_lowest = True)
    df['capital-loss'] = pd.cut(df['capital-loss'], [0, 1, 100000],
                   labels = ['No-Loss', 'Positive-Loss'], 
                   right = True, include_lowest = True)
    
    # Removing Spaces from the Data Entries
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)
    
    return df

In [ ]:
preprocessData(df)

In [ ]:
def apriori(df, min_sup):
    min_sup = (min_sup / 100.0) * len(df.index)
    print('Minimum Support Count: ', int(min_sup))
    
    # Generate Initial Set of Candidates
    print('\nPlease Wait. Calculating Frequent Itemsets.')
    candidates = getC1(df)
    
    # Generate Initial Set of Frequent Itemsets
    frequent = {k:v for k, v in list(candidates.items()) if v >= min_sup}
    freq = len(frequent) # No. of Frequent Itemsets Generated
    step = 1
    total = freq
    display(frequent, step)
    
    while 1:
        step += 1
        print('\n Calculating Frequent Itemsets.')
        candidates = join(frequent, step)
        frequent = prune(df, candidates, frequent, min_sup)
        freq = len(frequent)
        total += freq
        if freq == 0:
            break
        display(frequent, step)
        
    print('\nFrequent Itemsets Have Been Generated!')
    print('No. of Frequent Itemsets Generated: ', total)
    
def getC1(df):
    candidates = {}
    for index, row in df.iterrows():
        data = list(row)
        for item in data:
            if item in list(candidates.keys()):
                candidates[item] += 1
            else:
                candidates[item] = 1
    return candidates
        
def display(frequent, k):
    print('\n---- Frequent [', k, '] Itemsets ----')
    for i in frequent:
        key = i
        val = frequent[i]
        print(key, ' - ', val)
        
def join(frequent, k):
    cand_sets = []
    items = list(frequent.keys())
    size = len(items)
    for i in range(size):
        for j in range(i, size):
            joined = list(set(str(items[i]).split(' ')) | set(str(items[j]).split(' ')))
            itemset = list(itert.combinations(joined, k))
            if len(itemset) > 0:
                itemset = list(itemset[0])
                key = ' '.join(str(x) for x in itemset)
                cand_sets.append(key)
    return cand_sets

def sortInto(candidates):
    localCache = []
    for i in candidates:
        key = i.split(' ')
        key.sort()
        temp = tuple(key)
        if temp not in localCache:
            localCache.append(temp)
    localCache = [' '.join(item) for item in localCache]
    return localCache

def prune(df, candidates, frequent, min_sup):
    freq_sets = {}
    candidates = sortInto(candidates)
    for row in df.iterrows():
        index, data = row
        dataset = set(data)
        for i in candidates:
            itemset = i.split()
            if has_infrequent_subset(i, frequent) == 0:
                itemset_set = set(itemset)
                diff = itemset_set - dataset
                if len(diff) == 0:
                    if i not in freq_sets:
                        freq_sets[i] = 1
                    else:
                        freq_sets[i] += 1
    for key in list(freq_sets.keys()):
        if freq_sets[key] < min_sup:
            del freq_sets[key]
    return freq_sets

def has_infrequent_subset(itemset, frequent):
    itemset = itemset.split()
    k = len(itemset)
    subsets = list(itert.combinations(itemset, k-1))
    freq_sets = [set(str(x).split()) for x in list(frequent.keys())]
    for i in subsets:
        if set(i) not in freq_sets:
            return 1
    return 0



In [ ]:
def main():
    min_sup = eval(input('Support Threshold %: '))
    start_time = time.time()
    apriori(df, min_sup)
    run_time = time.time() - start_time
    print('\nExecution Time: ', run_time)
main()

## FP-Growth

In [ ]:
class Tree:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
        self.children = []
        
    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.value)+repr(self.weight)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret
    
    def isContainedIn(self, obj):
        for item in obj.children:
            if self.value == item.value:
                return item
        return False
finalList = []


def fpg(df, min_sup):
    size = len(df.index)
    min_sup = (min_sup / 100.0) * size
    print('Minimum Support Count: ', int(min_sup))
    
    # Generate Initial Set of Candidates
    candidates = getC1(df)
    
    # Generate Initial Set of Frequent Itemsets
    frequent = {k:v for k, v in list(candidates.items()) if v >= min_sup}
    print('Database Scan Complete.')
    
    # Generate List of Frequent Itemsets in Descending Order
    flist = sorted(list(frequent.items()), key=operator.itemgetter(1), reverse=True)
    itemsets1 = flist
    flist = [i[0] for i in flist]
    print('\nF-List Generated: ', flist)
    print('\nPlease Wait. Scanning Database Second Time.')
    df = df.reset_index()
        
    # Scan Database Again to Find Ordered Frequent Itemlist
    ordFreqList = [[] for _ in range(len(df))]
    for i, j in df.iterrows():
        if i>0 and i % 10000 == 0:
            print('Read', i, 'Records.')
        row = j.tolist()
        for item in flist:
            if item in row:
                ordFreqList[i].append(item)
    print('Database Scan Complete.')
    display(itemsets1, 1)
    freqItemSets = frequentItems(flist, ordFreqList, min_sup)
    total = len(itemsets1) + len(freqItemSets)
    dictlist = []
    maxi = 0
    for key, value in freqItemSets.items():
        if len(key) > maxi:
            maxi = len(key)
        temp = [key,value]
        dictlist.append(temp)
    for i in range(2, maxi+1):
        display(dictlist, i)
    
    print('\nFrequent Itemsets Have Been Generated!')
    print('No. of Frequent Itemsets Generated: ', total)

def getC1(df):
    candidates = {}
    for index, row in df.iterrows():
        data = list(row)
        if index>0 and index % 10000 == 0:
            print('Read', index, 'Records.')
        for item in data:
            if item in list(candidates.keys()):
                candidates[item] += 1
            else:
                candidates[item] = 1
    return candidates
        
def display(sets, k):
    print('\n---- Frequent [', k, '] Itemsets ----')
    if k == 1:
        for i in sets:
            key = list(str(i[0]).split())
            val = i[1]
            print(key, ' - ', val)
    else:
        for i in sets:
            if len(i[0]) == k:
                key = list(i[0])
                val = i[1]
                print(key, ' - ', val)            

# Generate the FP Tree
def makeTree(flist):
    init = Tree([], 0)
    for item in flist:
        node = init
        for elem in item:
            newnode = Tree(elem, 1)
            index = newnode.isContainedIn(node)
            if index:
                index.weight += 1
                node = index
            else:
                node = node.children
                node.append(newnode)
                node = newnode
    return init

# Generate the Frequent Pattern Base
def getPaths(tree, item, lst = []):
    node = tree
    if node.value == item:
        lst.append(node.weight)
        temp = []
        temp.extend(lst)
        temp.pop(0)
        finalList.append(temp)
        lst.pop()
    else:
        lst.append(node.value)
        for child in node.children:
            getPaths(child, item)
        lst.pop()

# Generate Frequent Itemsets        
def frequentItems(items, flist, min_sup):
    frequent_patterns = {}
    global finalList
    tree = makeTree(flist)
    for item in items:
        getPaths(tree, item)
        condPattBase = finalList
        localDict = freqPatterns(item, condPattBase, min_sup)
        frequent_patterns.update(localDict)
        finalList = []
    return frequent_patterns
        
def freqPatterns(item, patterns, min_sup):
    globalDict = {}
    for pattern in patterns:
        localDict = {}
        if len(pattern) > 1:
            elements = getCombs(pattern[:-1])
            for elem in elements:
                elem.append(item)
                tup = tuple(elem)
                elem.pop()
                if tup not in list(globalDict.keys()):
                    count = getCount(elem, patterns)
                    if count >= min_sup:
                        localDict[tup] = count
        globalDict.update(localDict)
    return globalDict

def getCombs(x):
    n = len(x)
    c = [list(comb) for i in range(n) for comb in combinations(x, i + 1)]
    return c

def getCount(elem, patterns):
    support = 0
    for pattern in patterns:
        flag = 1
        for ele in elem:
            if ele not in pattern:
                flag = 0
        if flag == 1:
            support += pattern[-1]
    return support


In [ ]:
def main():
    min_sup = eval(input('Support Threshold %: '))
    start_time = time.time()
    fpg(df, min_sup)
    run_time = time.time() - start_time
    print('\nExecution Time: ', run_time)

main()

# ---------------------------------------------------------------------------------------------------------------

# new method

# Cross Validation Kfolds

In [ ]:
def cross_validation(df, k):
    dataset_size = df.shape[0]
    fold_size = int(dataset_size / k)
    folds = []
    first = 0
    last = fold_size
    for i in range(k):
        folds.append(data_frame.iloc[first:last])
        first += fold_size
        last += fold_size
    
    return folds